In [3]:
import cv2
import numpy as np

In [75]:
stop_sign_img = cv2.imread('../../data/stop_sign/two_stop_signs.jpg')
stop_sign_img = cv2.resize(stop_sign_img, (500, 500))
frames = [stop_sign_img]

In [76]:
stop_sign_reference = cv2.imread('../../data/reference_signs/stop_sign.jpg')
stop_sign_reference.shape

(500, 500, 3)

In [ ]:
#TODO: Use the method to read the video file (slow_traffic_small.avi)
videopath = "../data/slow_traffic_small.mp4"  # Path to the video file
# frames, frame_width, frame_height, frame_rate = read_video(videopath)

#TODO: Create the Kalman filter object
kf = cv2.KalmanFilter(4, 2)
#TODO: Initialize the state of the Kalman filter
kf.measurementMatrix =  np.array([[1,0,0,0],[0,1,0,0]], np.float32) # Measurement matrix np.array of shape (2, 4) and type np.float32
# en la matriz de measuremente indicamos que unicamente estamos midiendo las posiciones x e y
dt = 50
kf.transitionMatrix = np.array([[1,0,dt,0],[0,1,0,dt],[0,0,1,0],[0,0,0,1]], np.float32) # Transition matrix np.array of shape (4, 4) and type np.float32
kf.processNoiseCov = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], np.float32) *0.03 # Process noise covariance np.array of shape (4, 4) and type np.float32

measurement = np.array((2, 1), np.float32)
prediction = np.zeros((2, 1), np.float32)

#TODO: Show the frames to select the initial position of the object
x,y,w,h = 0,0,500,500
track_window = (x, y, w, h)
#TODO: Compute the center of the object
cx = x + w // 2
cy = y + h // 2
#TODO: Initialize the state of the Kalman filter
kf.statePost = np.array([[cx], [cy], [0], [0]], np.float32)

# Initialize the covariance matrix
kf.errorCovPost = np.eye(4, dtype=np.float32)

#Predict the position of the object
prediction = kf.predict()
print(prediction)

#TODO: Update the measurement and correct the Kalman filter
measurement = np.array([[cx], [cy]], np.float32) #en los none introducimos las coordenadas x e y medidas
kf.correct(measurement)

#TODO: Crop the object 
crop = stop_sign_reference[y:y+h, x:x+w].copy() 
#TODO: Convert the cropped object to HSV
hsv_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
#TODO: Compute the histogram of the cropped object (Reminder: Use only the Hue channel (0-180))
mask=cv2.inRange(hsv_crop,(0,60,32), (180,255,255))

crop_hist = cv2.calcHist([hsv_crop], [0], mask=mask, histSize=[15], ranges=[0, 180])
cv2.normalize(crop_hist, crop_hist, 0, 255, cv2.NORM_MINMAX)

print(f'Initial position selected: {x}, {y}')

[[250.]
 [250.]
 [  0.]
 [  0.]]
Initial position selected: 0, 0


In [78]:
# crop_hist

In [79]:
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 1)
i=0
# x, y, w, h = 300, 200, 200, 200 # simply hardcoded the values
# track_window = (x, y, w, h)
frames = [stop_sign_img] * 100
for frame in frames[i:]:
    #TODO: Copy the frame 
    input_frame = frame.copy()
    #TODO: Convert the frame to HSV
    img_hsv = cv2.cvtColor(input_frame, cv2.COLOR_BGR2HSV)
    
    # Compute the back projection of the histogram
    img_bproject = cv2.calcBackProject([img_hsv], [0], crop_hist, [0, 180], 1)
    cv2.imshow('Backprojection', img_bproject)
    # cv2.waitKey(0)
    
    # Apply the mean shift algorithm to the back projection
    # print(track_window)
    # ret, track_window = cv2.meanShift(img_bproject, track_window, term_crit)
    ret, track_window = cv2.CamShift(img_bproject, track_window, term_crit)
    x_,y_,w_,h_ = track_window
    #TODO: Compute the center of the object
    c_x = x_ + w_ // 2
    c_y = y_ + h_ // 2
    
    # Predict the position of the object
    prediction = kf.predict()

    #TODO: Update the measurement and correct the Kalman filter
    measurement = np.array([[c_x], [c_y]], np.float32)
    kf.correct(measurement)
    
    # Draw the predicted position
    cv2.circle(input_frame, (int(prediction[0][0]), int(prediction[1][0])), 5, (0, 0, 255), -1)
    cv2.circle(input_frame, (int(c_x), int(c_y)), 5, (0, 255, 0), -1)
    cv2.rectangle(input_frame, (x_, y_), (x_+w_, y_+h_), (255, 0, 0), 2)
    # Show the frame with the predicted position
    cv2.imshow('Frame', input_frame)
    key = cv2.waitKey(0)
    if key == ord('q'):
        break

cv2.destroyAllWindows()